In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from datetime import date 
import datetime
from dateutil.relativedelta import relativedelta
from datetime import timedelta
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
import itertools
import math

from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [ ]:
data =  pd.read_csv("demo_try.csv", parse_dates=True)
data.head()

In [ ]:
data_new = data.loc[:,['Name','Date','Quantity','Item : Internal ID']]
data_new.rename(columns={'Item : Internal ID': 'Product_Id', 'Name':'Customer_Name'}, inplace=True)

In [ ]:
result = data_new.groupby(['Customer_Name', 'Product_Id', 'Date', 'Quantity']).size().reset_index()
# result = result.count() #result['Customer_Name'].value_counts()
result.head(6)

In [ ]:
all_customer = set(data_new['Customer_Name'])

In [ ]:
def lock_customer_details(customer_name, result):
    data_final = result[result['Customer_Name'] == customer_name]
    return data_final

def prod_wise_customer(prod_wise_cus, gr_wise_pr):
    final_ = prod_wise_cus.get_group(gr_wise_pr)
    final_ = final_.loc[:,['Date','Quantity']]
    final_["Date"]= pd.to_datetime(final_["Date"], dayfirst=True)
    final_.set_index('Date', inplace=True)
    final_.fillna(0,inplace=True)
    final_ =final_['Quantity'].resample('D').sum()
    print(final_)

In [ ]:
for customer in all_customer:
    customer_info = lock_customer_details(customer, result)
    prod_wise_cus = customer_info.groupby(['Product_Id'])
    for gr_wise_pr in prod_wise_cus.groups.keys():
        final_df = prod_wise_customer(prod_wise_cus, gr_wise_pr)
        print(final_df)
        break
    # print(customer_info)
    break

In [ ]:
def Simple_Exponential_Smooting(train_data, n_period):
    model_simple_es = SimpleExpSmoothing(train_data).fit()
    predictions_simple_es = model_simple_es.forecast(n_period)
    test_pred_ses = predictions_simple_es[-n_period:]
    predictions_ses_final = test_pred_ses.mask(test_pred_ses<0,0).apply(np.ceil)
    return predictions_ses_final

In [ ]:
print(Simple_Exponential_Smooting(final_df, 1))

In [ ]:
import numpy as np
from datetime import datetime, timedelta
from hmmlearn import hmm

In [ ]:
# Hidden Markov Model initialization
model = hmm.MultinomialHMM(n_components=10)

In [ ]:
X = [[5, 9, 14, 55], [66, 9, 15, 5]]

In [ ]:
# Fit the model to the data
model.fit(X)

In [ ]:
next_purchase = model.predict([[66, 9, 15, 5]])  # Use February data as input
print(f"Predicted next purchase: Product {next_purchase[0]}")